In [1]:
from scipy.spatial.distance import directed_hausdorff

import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
import nrrd

In [6]:
## evaluate man HD95 (hausdorff distance) and dice score

def dice_coefficient(prediction, labels):
    # measures the overlap between the predicted segmentation and the ground truth
    intersection = np.sum(prediction * labels)
    union = np.sum(prediction) + np.sum(labels)
    dice = (2.0 * intersection) / (union + 1e-6)
    return dice


def hausdorff_metric(prediction, labels):
    # quantifies the maximum discrepancy between the predicted segmentation and the ground truth
    distances = []
    for i in range(prediction.shape[2]):
        d1 = directed_hausdorff(prediction[:, :, i], labels[:, :, i])[0]
        d2 = directed_hausdorff(labels[:, :, i], prediction[:, :, i])[0]
        distances.extend([d1, d2])
    distances.sort()
    percentile_95 = distances[int(0.95 * len(distances))]
    return percentile_95

In [2]:
# image, _ = nrrd.read("../../data/ASOCA/Normal/CTCA/Normal_17.nrrd")

truth_path = "../../data_flat/Annotation_Normal_17.nrrd"
nrrd_data, nrrd_header = nrrd.read(truth_path)
lbl = np.array(nrrd_data)

prediction_path = "../../project/model1_auto3dseg/segresnet_0/prediction_testing/CTCA_Normal_17.nii.gz"
prediction_nib = nib.load(prediction_path)
pred = np.array(prediction_nib.dataobj)

In [8]:
mean_dice = np.mean([dice_coefficient(pred[:, :, i], lbl[:, :, i]) for i in range(pred.shape[2])])
mean_dice       # higher = better, best = 0.89 (ASOCA Challenge)

0.4395590417257778

In [9]:
hausdorff_95 = hausdorff_metric(pred, lbl)
hausdorff_95    # lower = better, best = 1.89 (ASOCA Challenge)

2.8284271247461903